# Задача о монетах

Вася рассказал Мише, что в его копилке сейчас n монет с суммарной стоимостью m рублей. Миша предполагает, что в копилке Васи могли быть только монеты номиналом 1, 5 или 10 рублей. Помогите ему узнать, могло ли такое быть.

### Входные данные
Первая строка содержит число n (1 ≤ n ≤10000), вторая строка содержит число m (1 ≤ m ≤ 100000).

### Выходные данные
Если у Миши могло быть n монет с суммарной стоимостью m рублей, выведите три числа: число монет номиналом 1, 5 и 10 рублей, соответственно. Если есть несколько решений, выведите любое. Если такая ситуация невозможна, выведите одно число −1.

Примечание: будем считать, что наличие монет каждого номинала является обязательным условием.

### Проверочные данные

входные данные

6
37

выходные данные

2 1 3

входные данные

5
42

выходные данные

-1

##  Вариант решения №1. Брутфорс

Перебираем все возможные варианты комбинаций монет в копилке. Не подходит для больших n и m.

In [7]:
from itertools import permutations

In [24]:
m, n = 37, 6
horizon = [10]*min(n, (m-6)//10) + [5]*min(n, (m-11)//5) + [1]*min(n, (m-16))
horizon

[10, 10, 10, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1]

In [33]:
%time print(*next(([x.count(y) for y in (1, 5, 10)] for x in
                                        set(permutations(horizon, r=n)) if sum(x) == m), [-1]))

2 1 3
CPU times: user 205 ms, sys: 9.64 ms, total: 214 ms
Wall time: 306 ms


## Вариант решения №2. Динамическое программирование
Проходим по всему «горизонту» номиналов и проверяем на соответствие условию.

In [32]:
b, k = [0] * len(horizon), 0

for i in range(len(b)):
    if sum(b) <= m - horizon[i] and k < n:
        b[i] = horizon[i]
        k += 1

%time print(sum(b) == m and ' '.join(map(str, (b.count(y) for y in (1, 5, 10)))) or -1)

2 1 3
CPU times: user 116 µs, sys: 90 µs, total: 206 µs
Wall time: 143 µs


## Вариант решения №3. Дерево глубины m

Корень дерева — сумма стоимостей монет, глубина — количество монет. Ветвление — вычетание из узла каждого номинала из трех. 

In [118]:
import numpy as np
from anytree import Node, RenderTree
from typing import List

In [128]:
def descent_tree(node: Node) -> List[Node]:
    #node_amount = int(node.name.rsplit('/', 1)[-1])
    node_amount = int(node.name)
    if node_amount > 0 and node.depth < n:
        #for branch in (Node(f"{node.name}/{node_amount - x}", parent=node) for x in (10, 5, 1)):
        for branch in (Node(f"{node_amount - x}", parent=node) for x in (10, 5, 1)):
            descent_tree(branch)
    else:  
        if node_amount == 0:
            null_leafs.append(node)
    return null_leafs

In [135]:
root = Node(str(m))
null_leafs = descent_tree(root)

res, k = [0], 0
parent = null_leafs[0].parent
while True:
    if parent == None:
        break
    else:
        res += [int(f'{parent.name}'.rsplit('/', 1)[-1])]
        parent = parent.parent
        k += 1

%time print(null_leafs != [] and ' '.join(map(str, (list(np.diff(res)).count(y) for y in (1, 5, 10)))) or -1)

2 1 3
CPU times: user 339 µs, sys: 110 µs, total: 449 µs
Wall time: 385 µs


In [130]:
for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))

37
├── 27
│   ├── 17
│   │   ├── 7
│   │   │   ├── -3
│   │   │   ├── 2
│   │   │   │   ├── -8
│   │   │   │   ├── -3
│   │   │   │   └── 1
│   │   │   │       ├── -9
│   │   │   │       ├── -4
│   │   │   │       └── 0
│   │   │   └── 6
│   │   │       ├── -4
│   │   │       ├── 1
│   │   │       │   ├── -9
│   │   │       │   ├── -4
│   │   │       │   └── 0
│   │   │       └── 5
│   │   │           ├── -5
│   │   │           ├── 0
│   │   │           └── 4
│   │   ├── 12
│   │   │   ├── 2
│   │   │   │   ├── -8
│   │   │   │   ├── -3
│   │   │   │   └── 1
│   │   │   │       ├── -9
│   │   │   │       ├── -4
│   │   │   │       └── 0
│   │   │   ├── 7
│   │   │   │   ├── -3
│   │   │   │   ├── 2
│   │   │   │   │   ├── -8
│   │   │   │   │   ├── -3
│   │   │   │   │   └── 1
│   │   │   │   └── 6
│   │   │   │       ├── -4
│   │   │   │       ├── 1
│   │   │   │       └── 5
│   │   │   └── 11
│   │   │       ├── 1
│   │   │       │   ├── -9
│   │   │       │   ├── -4
│   │   │       